# Example for usage ARc Policy via a tool for a Strands Agent

Install requirements

In [ ]:
%pip install -r requirements.txt

## Setup for the agent

In [ ]:
from strands import Agent
from strands.models import BedrockModel
import boto3
from botocore.config import Config as BotocoreConfig
from strands.telemetry import StrandsTelemetry
import logging

# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.DEBUG)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

# Setup tracing
StrandsTelemetry().setup_console_exporter()

In [ ]:
# Supply the pre-installed polciy and guardrail IDs
ARC_POLICY_ID = "xvi5i57kz3gl"
GUARDRAIL_ID = "xb8kiy3pe4qr"
GUARDRAIL_VERSION = "DRAFT"
# MODEL_ID = "us.anthropic.claude-sonnet-4-20250514-v1:0"
MODEL_ID = "us.amazon.nova-lite-v1:0"

In [ ]:
boto_config = BotocoreConfig(
    retries={"max_attempts": 3, "mode": "standard"},
    connect_timeout=5,
    read_timeout=60
)

# Create a Bedrock model with guardrail configuration
bedrock_model = BedrockModel(
    boto_client_config=boto_config,
    model_id=MODEL_ID,
    guardrail_id=GUARDRAIL_ID,
    guardrail_version=GUARDRAIL_VERSION,
    guardrail_trace="enabled_full",
    guardrail_redact_input=True,
    guardrail_redact_output=True,
    streaming=False,
)

# Create agent with the guardrail-protected model
auto_policy_agent = Agent(
    system_prompt="You are a helpful assistant that will answer questions related to auto insurance policy.  When processing the result, make sure to use the information from the Automated Reasoning Output to modify the result to be correct.  Make sure that the Automated Reasoning Output section contains the IDs for the Rules, Variables, and Types from Automated Reasoning.",
    model=bedrock_model,
)


In [ ]:
# Use the protected agent for conversations
response = auto_policy_agent("If I have a trailer and I get into an accident is that considered a separate vehicle?  Make sure that the output includes the automated reasoning output from the trace as well")

# Handle potential guardrail interventions
# if response.stop_reason == "guardrail_intervened":
#    print("Content was blocked by guardrails, conversation context overwritten!")